In [ ]:
# PLEASE BE PATIENT WITH THE FIRST BLOCK
# the kernel needs to load and netgen needs to be imported, which takes its time
from netgen.occ import *

In [ ]:
import piplite
await piplite.install("https://triadtitans.github.io/rigid_body_interactive/static/pyodide/lib_rigid_body-0.0.1-cp312-cp312-pyodide_2024_0_wasm32.whl")
await piplite.install("https://files.pythonhosted.org/packages/93/c1/68423f43bc95d873d745bef8030ecf47cd67f932f20b3f7080a02cff43ca/widgetsnbextension-4.0.11-py3-none-any.whl")
await piplite.install("pythreejs")

from lib_rigid_body.rigid_body_FEM import *
import lib_rigid_body.rigid_body_FEM.bla as bla

In [ ]:
import pythreejs as p3

from IPython.display import display
import ipywidgets as widgets

### Helper Functions for body setup

In [2]:
def extract_vertices(obj: TopoDS_Shape):
    "extracts a p3js compatible vertex list from a netgen.occ TopoDS_Shape"
    
    data = obj._webgui_data()["Bezier_trig_points"]
    
    # for every face, each of the verts arrays holds one vertex
    verts1 = data[0]
    verts2 = data[1]
    verts3 = data[2]
    
    # corresponding normals
    normals1 = data[3]
    normals2 = data[4]
    normals3 = data[5]

    combined_vertices = []
    for i in range(0, len(verts1), 4):
        combined_vertices.append(verts1[i : i+3])
        combined_vertices.append(verts2[i : i+3])
        combined_vertices.append(verts3[i : i+3])

    combined_normals = []
    for i in range(0, len(normals1), 3):
        combined_normals.append(normals1[i : i+3])
        combined_normals.append(normals2[i : i+3])
        combined_normals.append(normals3[i : i+3])
            
    return combined_vertices, combined_normals


def body_from_solid(obj):
    "extracts the mass matrix of the TopAbs_ShapeEnum.SOLID obj, using the figures computed by netgen"

    # important: move the center of mass into the origin
    obj = obj.Move((-obj.center[0], -obj.center[1], -obj.center[2]))
    
    # copy the inertia matrix from netgen
    inertia_matrix = bla.Matrix(3,3)
    for i in range(3):
        for j in range(3):
            inertia_matrix[i, j] = obj.inertia[i, j]

    # copy the center of mass from netgen
    center_of_mass = bla.Vector(3)
    for i in range(3): center_of_mass[i] = obj.center[i]

    # rearrange it in C++ to make the mass matrix (the elegant way, using MatrixView)
    body = RigidBody_FEM()
    for i in range(3) : body.center[i] = obj.center[i]
    body.mass = obj.mass
    body.inertia = inertia_matrix
    body.recalcMassMatrix()

    body.vertices, body.normals = extract_vertices(obj)
    
    return body

## Setup System

In [3]:
# set up OCC CAD model
# center of mass is automatically moved to origin by body_from_solid
box = Box(Pnt(-0.5,-0.5,-0.5), Pnt(0.5,0.5,0.5))

In [4]:
# set up physics simulation environment
rbs = RBS_FEM()
rbs.gravity = (0, 0, 0)

# set up physics simulation object for cube
box =  body_from_solid(box)

rbs.addBody(box)

rbs.saveState()

## Setup Graphics

In [5]:
view_width = 1300
view_height = 500
buffergeos = []
p3meshes = []

# set up pythreejs 3d objects
for body in rbs.bodies():
    buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(body.vertices), "normal" : p3.BufferAttribute(body.normals)})
    material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
    p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))
    buffergeos.append(buffergeom)
    p3meshes.append(p3mesh)

# extra scene contents
camera = p3.PerspectiveCamera( position=[5, 3, 5], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()
grid = p3.GridHelper(500, 500//5, "#2F4F4F", "#2F4F4F")
axesHelper = p3.AxesHelper(5)

# set up scene
scene = p3.Scene(children=[camera, key_light, ambient_light, grid, axesHelper, *p3meshes])
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [6]:
for m in p3meshes:
    m.matrixAutoUpdate = False # make mesh movable

def refresh():
    "updates all pythreejs object transformations"
    for i in range(0, len(rbs.bodies())):
        p3meshes[i].matrix=rbs.bodies()[i].transformation.asTuple()

## Renderer

In [7]:
display(renderer)
refresh()

Renderer(camera=PerspectiveCamera(aspect=2.6, position=(5.0, 3.0, 5.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, …

### reset transformation

In [8]:
box = rbs.bodies()[0]
trafo = Transformation()
box.transformation = trafo
refresh()

### rotation in degrees

In [9]:
trafo = Transformation()
trafo.setRotationDeg(0, 10)
box.transformation = trafo
refresh()
print(trafo.angle_deg)

9.999999999999975


### applying another transformation: SE(3) is not commutative

In [10]:
trafo = Transformation()
trafo2 = Transformation()

trafo.setRotationDeg(0, 10)
trafo.setTranslation(0, 0, -10)

trafo2.setRotationDeg(1, -10)
trafo2.setTranslation(0, 0, 10)

box.transformation = trafo*trafo2 # exchange trafo and trafo2 to see the effect
refresh()